# Import Libraries

In [ ]:
import numpy as np 
import pandas as pd
import datetime as dt
import io

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from lifetimes.plotting import plot_period_transactions
import plotly.graph_objs as go
from plotly.offline import iplot
from operator import attrgetter
import squarify

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

from IPython.display import Markdown

def bold(string):
    display(Markdown(string))

import warnings
warnings.filterwarnings('ignore')

# Load Data

In [ ]:
df = pd.read_csv("/mnt/hdd/Datasets/online_retail_II.csv")
df.head()

In [ ]:
def df_stats(data):
    bold("**" + " SHAPE ".center(50, "#") + "**")
    print("ROWS: {}".format(data.shape[0]))
    print("COLUMNS: {}".format(data.shape[1]))
    bold("**" + " TYPES ".center(50, "#") + "**")
    print(data.dtypes)
    bold("**" + " MISSING VALUES ".center(50, "#") + "**")
    print(data.isnull().sum())
    bold("**" + " DUPLICATED VALUES ".center(50, "#") + "**")
    print("NUMBER OF DUPLICATED VALUES: {}".format(data.duplicated().sum()))
    bold("**" + " MEMORY USAGE ".center(50, "#") + "**")
    buf = io.StringIO()
    data.info(buf=buf)
    info = buf.getvalue().split("\n")[-2].split(":")[1].strip()
    print("Memory Usage: {}".format(info))
    bold("**" + " DESCRIBE ".center(50, "#") + "**")
    print(data.describe().T)

In [ ]:
df_stats(df)

# Outlier Detection

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

# Data Preprocess

In [ ]:
df.dropna(inplace=True)

In [ ]:
df = df[~df["Invoice"].astype(str).str.contains("C", na=False)]
df.info()

In [ ]:
df["InvoiceDate"] = df["InvoiceDate"].apply(pd.to_datetime)

# Cohort Analysis

In [ ]:
df["InvoiceMonth"] = df["InvoiceDate"].dt.to_period("M")

In [ ]:
df["Cohort"] = df.groupby("Customer ID")["InvoiceDate"].transform("min").dt.to_period("M")

In [ ]:
df_cohort = df.groupby(['Cohort', 'InvoiceMonth'], as_index=False).agg(n_customers=('Customer ID', 'nunique')).reset_index(drop=False)

In [ ]:
df_cohort.head()

In [ ]:
df_cohort["period"] = (df_cohort["InvoiceMonth"] - df_cohort["Cohort"]).apply(attrgetter("n"))

In [ ]:
df_cohort.head()

In [ ]:
cohort_pivot = df_cohort.pivot_table(index="Cohort", columns="period", values="n_customers")
cohort_pivot

In [ ]:
cohort_size = cohort_pivot.iloc[:, 0]
cohort_size

In [ ]:
retention_matrix = cohort_pivot.divide(cohort_size, axis=0)

In [ ]:
with sns.axes_style("white"):
    fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True, gridspec_kw={"width_ratios": [1, 11]})
    sns.heatmap(retention_matrix, mask=retention_matrix.isnull(), annot=True, cbar=False, fmt=".0%", cmap="coolwarm", ax=ax[1])
    ax[1].set_title("Monthly Cohorts: User Retention", fontsize=14)
    ax[1].set(xlabel="# of periods", ylabel="")
    white_cmap = mcolors.ListedColormap(["white"])
    sns.heatmap(pd.DataFrame(cohort_size).rename(columns={0: "cohort_size"}),annot=True,cbar=False,fmt="g",cmap=white_cmap,ax=ax[0])
    fig.tight_layout()

# RFM Analysis

In [ ]:
df["InvoiceDate"].max()

In [ ]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [ ]:
today_date = dt.datetime(2011, 12, 10)

In [ ]:
rfm = df.groupby("Customer ID", as_index=False).agg({
    "InvoiceDate": lambda x: (today_date - x.max()).days,
    "Invoice": lambda x: x.nunique(),
    "TotalPrice": lambda x: x.sum()
})

rfm.columns = ["Customer ID", "Recency", "Frequency", "Monetary"]
rfm = rfm[rfm["Monetary"] > 0]
rfm.head()

In [ ]:
rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1])
rfm["FrequencyScore"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"], 5, labels=[1,2,3,4,5])
rfm["RFM"] = rfm["RecencyScore"].astype(str) + rfm["FrequencyScore"].astype(str)

In [ ]:
rfm.head()

# CLV

In [ ]:
cltv = df.groupby("Customer ID", as_index=False).agg({
    "Invoice": lambda x: x.nunique(),
    "Quantity": lambda x: x.sum(),
    "TotalPrice": lambda x: x.sum()
})

cltv.columns = ["Customer ID", "TotalTransaction", "TotalUnit", "TotalPrice"]
cltv.head()

In [ ]:
cltv["AverageOrderValue"] = cltv["TotalPrice"] / cltv["TotalTransaction"]
cltv["PurchaseFrequency"] = cltv["TotalTransaction"] / cltv.shape[0]
repeat_rate = cltv[cltv["TotalTransaction"] > 1].shape[0] / cltv.shape[0]
churn_rate = 1 - repeat_rate
cltv["ProfitMargin"] = cltv["TotalPrice"] * 0.1
cltv["CustomerValue"] = cltv["AverageOrderValue"] * cltv["PurchaseFrequency"]
cltv["CLV"] = (cltv["CustomerValue"] / churn_rate) * cltv["ProfitMargin"]
cltv["Segment"] = pd.qcut(cltv["CLV"], 4, labels=["class D", "class C", "class B", "classA"])
cltv.groupby("Segment", as_index=False).agg({"count", "mean", "sum"})